### **회귀분석 **
-------
    수치형 변수 간의 Input 원인과 Output 간의 관곌르 추정하거나 예측하는데 사용한다.
    변수 간의 강도와 방향을 측정하는 방법에는 상관 계수가 있다.

#### 상관계수의 종류
---
    상관계수 : 두 변수 간의 선형적인 관계가 어느정도 강한지를 나타낸다.
    - -1<= r <= 1
    - r이 1에 가까울수록 양의 선형관계를 의미한다.
    - r이 -1에 가까울수록 음의 선형관계를 의미한다.
    - r이 0 에 가까울수록 약한 선형 관계를 의미한다.


1. 상관계수
```
pandas.DataFrame.corr(method = 'pearson' , numeric_only = False)
```
- 상관관계 method
  - pearson
  - kendall
  - spearman
- numeric_only : 숫자 자료형만 포함(기본값 False)
  

In [7]:
import pandas as pd

# 주어진 데이터
data = {
    '키': [150, 160, 170, 175, 165],
    '몸무게': [42, 50, 70, 64, 56]
}
df = pd.DataFrame(data)

In [8]:
# 상관계수 계산
correlation = df.corr()

print(correlation)

            키       몸무게
키    1.000000  0.919509
몸무게  0.919509  1.000000


In [9]:
print(correlation.iloc[0,1])


0.9195090879163764


In [10]:
print(df['키'].corr(df['몸무게']))

0.9195090879163765


In [11]:
print(df.corr(method='spearman'))

       키  몸무게
키    1.0  0.9
몸무게  0.9  1.0


In [12]:
from scipy import stats

print(stats.pearsonr(df['몸무게'],df['키']))
print(stats.kendalltau(df['몸무게'],df['키']))
print(stats.spearmanr(df['몸무게'],df['키']))

PearsonRResult(statistic=0.9195090879163766, pvalue=0.027079456895589476)
SignificanceResult(statistic=0.7999999999999999, pvalue=0.08333333333333333)
SignificanceResult(statistic=0.8999999999999998, pvalue=0.03738607346849874)


## 단순 선형 회귀 분석
    2개 이상의 변수 간의 관계를 파악하려는 통계적 분석 방법이다.
    그중 단순 선형 회귀 분석은 하나의 독립변수로부터 하나의 종속변수와의 관계를 분석하거나 예측하는 방법이다.
    <예시> 키워드 광고를 많이 하면 판매량이 어떻게 변할까요?
    - 독립변수 : 원인을 나타내는 데이터다. 예) 키워드 광고
    - 종속변수 : 결과를 나타내는 데이터다. 예) 판매량

1. 최소제곱 선형 회귀 모델을 구축
```
ols('종속변수 ~ 독립변수',data=df).fit()
```
2. 제곱합

|   용어         |  | 의미 |
|--------------|------|--------|
| SST    | 총 제곱합   | 각 관측치가 평균으로부터 얼마나 떨어져 있는지를 나타냄     |
| SSR | 회귀 제곱합   | 회귀선이 평균으로부터 얼마나 떨어져 있는지를 나타냄     |
| SSE |오차 제곱합(잔차 제곱합) |관측치가 회귀선으로부터 얼마나 떨어져 있는지를 나타냄 |

3. 결정계수(R-squared)
결정 계수는 모델이 그 데이터를 얼마나 잘 설명하는지 나타내는 통계 값이다.
R^2 = SSR / SST , 이 값이 1에 가까울수록 잘 설명한다.

Q. 다음은 20명의 키와 몸무게에 관한 정보다. 이 데이터를 바탕으로 회귀 모델을 구축하고 각 소문제의 값을 구하시오.

In [13]:
import pandas as pd

data = {
    '키': [150, 160, 170, 175, 165, 155, 172, 168, 174, 158,
          162, 173, 156, 159, 167, 163, 171, 169, 176, 161],
    '몸무게': [42, 50, 70, 64, 56, 48, 68, 60, 65, 52,
            54, 67, 49, 51, 58, 55, 69, 61, 66, 53]
}
df = pd.DataFrame(data)

In [14]:
from statsmodels.formula.api import ols
model = ols('키 ~몸무게',data=df).fit()
print(model.summary())
print("결정계수:",model.rsquared)
print("기울기: ",model.params['몸무게'])
print('절편:',model.params['Intercept'])
print("p-values:",model.pvalues['몸무게'])
new_Data = pd.DataFrame({'몸무게':[67]})
result = model.predict(new_Data)
print('몸무게가 67일때 예측 키',result[0])
# 회귀 모델의 잔차 제곱합
df['잔차'] = df['키'] - model.predict(df)
print("잔차 제곱합 : ",sum(df['잔차']**2))
print("MSE:",(df['잔차']**2).mean())

                            OLS Regression Results                            
Dep. Variable:                      키   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     148.0
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           4.04e-10
Time:                        05:39:29   Log-Likelihood:                -45.761
No. Observations:                  20   AIC:                             95.52
Df Residuals:                      18   BIC:                             97.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    115.0676      4.158     27.671      0.0

In [15]:
from sklearn.metrics import mean_squared_error
pred = model.predict(df['몸무게'])
mse = mean_squared_error(df['키'],pred)
print('MSE : ',mse)

MSE :  5.687113319442214


몸무게의 95% 신뢰 구간을 구하시오.

In [16]:
print("신뢰구간:\n",model.conf_int(alpha=0.05).loc['몸무게'])


신뢰구간:
 0    0.716337
1    1.015351
Name: 몸무게, dtype: float64


In [18]:
new_data = pd.DataFrame({"몸무게":[50]})
pred = model.get_prediction(new_data)
result = pred.summary_frame(alpha=0.05)
print("예측값의 신뢰구간과 예측구간\n",result)


예측값의 신뢰구간과 예측구간
          mean   mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  158.359833  0.794986      156.68963     160.030037    152.820798   

   obs_ci_upper  
0    163.898869  


## 다중 선형 회귀 분석
    2개 이상의 독립변수와 하나의 종속변수 간 관계를 분석하는 방법이다.
    

Q. 다음은 매출액, 광고비, 직원 수에 관한 데이터다. 광고비와 직원 수는 독립변수고, 매출액은 종속변수다.<BR/>
 다중 선형 회귀 모델을 구축하고, 각 소문제의 값을 구하시오.

In [19]:
import pandas as pd
data = {
    '매출액': [300, 320, 250, 360, 315, 328, 310, 335, 326, 280,
            290, 300, 315, 328, 310, 335, 300, 400, 500, 600],
    '광고비': [70, 75, 30, 80, 72, 77, 70, 82, 70, 80,
            68, 90, 72, 77, 70, 82, 40, 20, 75, 80],
    '직원수': [15, 16, 14, 20, 19, 17, 16, 19, 15, 20,
            14, 5, 16, 17, 16, 14, 30, 40, 10, 50]
    }
df = pd.DataFrame(data)
print(df.head(3))

   매출액  광고비  직원수
0  300   70   15
1  320   75   16
2  250   30   14


1. 최소제곱법으로 다중 선형 회귀 모델을 구축하고 통계적 요약을 출력하시오.

In [20]:
from statsmodels.formula.api import ols
model = ols('매출액 ~ 광고비 +직원수 ',data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    매출액   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     8.907
Date:                Wed, 20 Nov 2024   Prob (F-statistic):            0.00226
Time:                        06:13:39   Log-Likelihood:                -108.22
No. Observations:                  20   AIC:                             222.4
Df Residuals:                      17   BIC:                             225.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    101.0239     71.716      1.409      0.1

In [21]:
print(df['광고비'].corr(df['매출액']))

0.13316981737040343


In [23]:
print("결정계수:",model.rsquared)

결정계수: 0.5116964327009041


In [24]:
print(round(model.params))

Intercept    101.0
광고비            2.0
직원수            6.0
dtype: float64


절편 101, 광고비 2.0 직원수 6.0 기울기를 가지고 있다.

In [25]:
print(model.pvalues['광고비'])

0.03764350647695994


In [26]:
new_data = pd.DataFrame({"광고비":[50],"직원수":[20]})
result = model.predict(new_data)
print("새로운 데이터의 매출액:",result)

새로운 데이터의 매출액: 0    310.57033
dtype: float64


In [27]:
df['잔차'] = df['매출액'] - model.predict(df)
print("잔차의 제곱합: ",sum(df['잔차']**2))

잔차의 제곱합:  58686.178271561075


In [33]:
# 같은 결과 다른방식
print(sum(model.resid**2))

58686.178271561075


In [35]:
MSE = (df['잔차']**2).mean()
print("MSE:",MSE)

MSE: 2934.3089135780533


In [40]:
data =(model.resid**2).mean()

,0
0,299.812391
1,152.299829
2,129.748999
3,26.554837
4,880.247632
5,193.448907
6,175.399483
7,776.359971
8,75.427637
9,7251.057009


In [43]:
print('95% 신뢰구간\n',model.conf_int(alpha=0.05))

95% 신뢰구간
                    0           1
Intercept -50.283684  252.331429
광고비         0.116785    3.522069
직원수         2.912406    8.945105


In [46]:
new_data = pd.DataFrame({"광고비":[45],"직원수":[22]})

pred = model.get_prediction(new_data)
result = pred.summary_frame(alpha=0.05)
print("예측값의 신뢰구간, 예측구간\n",result)

예측값의 신뢰구간, 예측구간
          mean    mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  313.330707  22.502058     265.855514       360.8059     180.58875   

   obs_ci_upper  
0    446.072663  


## 범주형 변수
--------
    독립변수로 수치형 변수와 범주형 변수를 함께 사용할 때는 범주형 변수는 통상적으로 원-핫 인코딩과 같은 방법을 사용해 수치화 해야한다.

In [47]:
from google.colab import files
uploaded = files.upload()

Saving study.csv to study.csv


In [48]:
import pandas as pd
import statsmodels.api as sm
df = pd.read_csv('study.csv')
df.head(3)

,study_hours,material_type,score
0,71,강의,95
1,34,독학,63
2,91,도서,95


In [50]:
from statsmodels.formula.api import ols
model = ols('score ~ material_type + study_hours',data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     991.9
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           4.42e-72
Time:                        06:53:33   Log-Likelihood:                -238.89
No. Observations:                 100   AIC:                             485.8
Df Residuals:                      96   BIC:                             496.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              59.2111    